In [1]:
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import numpy as np
from linear_cca import linear_cca
from torch.utils.data import BatchSampler, SequentialSampler
from DeepCCAModels import DeepCCA
from main import Solver
from utils import load_data, svm_classify
try:
    import cPickle as thepickle
except ImportError:
    import _pickle as thepickle

import gzip
import numpy as np
torch.set_default_tensor_type(torch.DoubleTensor)

In [2]:
############
# Parameters Section

device = torch.device('cuda')
print("Using", torch.cuda.device_count(), "GPUs")

# the path to save the final learned features
save_to = './new_features.gz'

# the size of the new space learned by the model (number of the new features)
outdim_size = 10

# size of the input for view 1 and view 2
input_shape1 = 784
input_shape2 = 784

# number of layers with nodes in each one
layer_sizes1 = [1024, 1024, 1024, outdim_size]
layer_sizes2 = [1024, 1024, 1024, outdim_size]

# the parameters for training the network
learning_rate = 1e-3
epoch_num = 100
batch_size = 800

# the regularization parameter of the network
# seems necessary to avoid the gradient exploding especially when non-saturating activations are used
reg_par = 1e-5

# specifies if all the singular values should get used to calculate the correlation or just the top outdim_size ones
# if one option does not work for a network or dataset, try the other one
use_all_singular_values = True

# if a linear CCA should get applied on the learned features extracted from the networks
# it does not affect the performance on noisy MNIST significantly
apply_linear_cca = True

# end of parameters section
############

# Each view is stored in a gzip file separately. They will get downloaded the first time the code gets executed.
# Datasets get stored under the datasets folder of user's Keras folder
# normally under [Home Folder]/.keras/datasets/
data1 = load_data('./noisymnist_view1.gz')
data2 = load_data('./noisymnist_view2.gz')


Using 3 GPUs
loading data ...
loading data ...


In [3]:

# Building, training, and producing the new features by DCCA
model = DeepCCA(layer_sizes1, layer_sizes2, input_shape1,
                input_shape2, outdim_size, use_all_singular_values, reg_par, device=device).double()
solver = Solver(model, epoch_num, batch_size, learning_rate, reg_par, device=device)
train1, train2 = data1[0][0], data2[0][0]
val1, val2 = data1[1][0], data2[1][0]
test1, test2 = data1[2][0], data2[2][0]

solver.fit(train1, train2, val1, val2, test1, test2)

set_size = [0, train1.size(0), train1.size(0) + val1.size(0), train1.size(0) + val1.size(0) + test1.size(0)]
loss, outputs = solver.test(torch.cat([train1, val1, test1], dim=0), torch.cat([train2, val2, test2], dim=0), outdim_size, apply_linear_cca)



[ INFO : 2018-10-02 15:22:02,311 ] - DataParallel(
  (module): DeepCCA(
    (model1): MlpNet(
      (layers): ModuleList(
        (0): Sequential(
          (0): Linear(in_features=784, out_features=1024, bias=True)
          (1): Sigmoid()
        )
        (1): Sequential(
          (0): Linear(in_features=1024, out_features=1024, bias=True)
          (1): Sigmoid()
        )
        (2): Sequential(
          (0): Linear(in_features=1024, out_features=1024, bias=True)
          (1): Sigmoid()
        )
        (3): Linear(in_features=1024, out_features=10, bias=True)
      )
    )
    (model2): MlpNet(
      (layers): ModuleList(
        (0): Sequential(
          (0): Linear(in_features=784, out_features=1024, bias=True)
          (1): Sigmoid()
        )
        (1): Sequential(
          (0): Linear(in_features=1024, out_features=1024, bias=True)
          (1): Sigmoid()
        )
        (2): Sequential(
          (0): Linear(in_features=1024, out_features=1024, bias=True)
     

[ INFO : 2018-10-02 15:35:53,233 ] - Epoch 32/100 - time: 22.17 - train_loss: -2.4608 - val_loss: -2.5604

[ INFO : 2018-10-02 15:36:15,425 ] - Epoch 00033: val_loss did not improve from -2.62586
[ INFO : 2018-10-02 15:36:15,426 ] - Epoch 33/100 - time: 22.19 - train_loss: -2.4698 - val_loss: -2.5780

[ INFO : 2018-10-02 15:36:37,586 ] - Epoch 00034: val_loss did not improve from -2.62586
[ INFO : 2018-10-02 15:36:37,588 ] - Epoch 34/100 - time: 22.16 - train_loss: -2.4786 - val_loss: -2.6214

[ INFO : 2018-10-02 15:36:59,762 ] - Epoch 00035: val_loss improved from -2.62586 to -2.66784, saving model to checkpoint.model
[ INFO : 2018-10-02 15:37:05,173 ] - Epoch 35/100 - time: 27.58 - train_loss: -2.4869 - val_loss: -2.6678

[ INFO : 2018-10-02 15:37:27,354 ] - Epoch 00036: val_loss improved from -2.66784 to -2.66889, saving model to checkpoint.model
[ INFO : 2018-10-02 15:37:32,730 ] - Epoch 36/100 - time: 27.56 - train_loss: -2.4950 - val_loss: -2.6689

[ INFO : 2018-10-02 15:37:54,91

[ INFO : 2018-10-02 15:51:42,198 ] - Epoch 72/100 - time: 22.19 - train_loss: -2.7010 - val_loss: -2.7129

[ INFO : 2018-10-02 15:52:04,377 ] - Epoch 00073: val_loss did not improve from -2.72127
[ INFO : 2018-10-02 15:52:04,378 ] - Epoch 73/100 - time: 22.18 - train_loss: -2.7052 - val_loss: -2.7008

[ INFO : 2018-10-02 15:52:26,555 ] - Epoch 00074: val_loss did not improve from -2.72127
[ INFO : 2018-10-02 15:52:26,556 ] - Epoch 74/100 - time: 22.18 - train_loss: -2.7092 - val_loss: -2.6507

[ INFO : 2018-10-02 15:52:48,675 ] - Epoch 00075: val_loss did not improve from -2.72127
[ INFO : 2018-10-02 15:52:48,676 ] - Epoch 75/100 - time: 22.12 - train_loss: -2.7130 - val_loss: -2.6971

[ INFO : 2018-10-02 15:53:10,789 ] - Epoch 00076: val_loss did not improve from -2.72127
[ INFO : 2018-10-02 15:53:10,791 ] - Epoch 76/100 - time: 22.11 - train_loss: -2.7171 - val_loss: -2.6730

[ INFO : 2018-10-02 15:53:33,017 ] - Epoch 00077: val_loss did not improve from -2.72127
[ INFO : 2018-10-02 

Linear CCA started!


In [4]:
new_data = []
# print(outputs)
for idx in range(3):
    new_data.append([outputs[0][set_size[idx]:set_size[idx + 1], :], outputs[1][set_size[idx]:set_size[idx + 1], :], data1[idx][1]])
# Training and testing of SVM with linear kernel on the view 1 with new features
[test_acc, valid_acc] = svm_classify(new_data, C=0.01)
print("Accuracy on view 1 (validation data) is:", valid_acc * 100.0)
print("Accuracy on view 1 (test data) is:", test_acc*100.0)

training SVM...
Accuracy on view 1 (validation data) is: 97.09
Accuracy on view 1 (test data) is: 96.82


In [5]:
# Saving new features in a gzip pickled file specified by save_to
print('saving new features ...')
f1 = gzip.open(save_to, 'wb')
thepickle.dump(new_data, f1)
f1.close()
w

saving new features ...


In [6]:
d = torch.load('checkpoint.model')
solver.model.load_state_dict(d)
solver.model.parameters()

<generator object Module.parameters at 0x7fe6a6523ba0>